<a href="https://colab.research.google.com/github/upamasaki/Brain_LSTM/blob/main/mylstm_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# ライブラリ読み込み

In [ ]:
#!pip install joypy
import numpy as np
import os
import glob
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

import plotly.express as px
import plotly.graph_objects as go
#from joypy import joyplot for matplotlib
import tensorflow as tf
from tqdm import tqdm
#import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize, StandardScaler
from sklearn.model_selection import train_test_split, GroupKFold, KFold

tqdm.pandas()

#%matplotlib inline
%matplotlib

In [ ]:
if tf.device('/CPU:0'):
    print(tf.device('/CPU:0'))

# Settings

In [ ]:
# When Debug mode is true, this notebook use small train data.
debug_mode = False
debug_data_count = int(1000)

file_override = False

tqdm.pandas()

# folloiwng settig takes time due to plotly is so slow.
visualize = False  # [True | False]
visualize2 = False
visualize3 = False

# Use trained model. no run training. Just use trained model only.

use_trained_model = False

###################################################
# Path setting
#
google_mount_path   = '/content/drive'
google_PROJECT_dir  = 'MyDrive/2021/100_GoogleBrain_VentilatorPressurePrediction'
root_path           = '{}/{}'.format(google_mount_path, google_PROJECT_dir)
path                = '{}/{}'.format(root_path, 'datasets')
dataset_ver         = 'v3.1'
exp_name            = 'mylstm_v3.3_lite'
dataset_path        = '{}/{}/{}'.format(root_path, 'datasets', dataset_ver)
log_path            = '{}/{}/{}'.format(root_path, 'log', exp_name)
result_path         = '{}/{}/{}/{}'.format(root_path, 'log', exp_name, 'result')


###################################################
# Google Drive Setting
#
if(not os.path.exists(google_mount_path)):
  from google.colab import drive
  drive.mount(google_mount_path)

###################################################
# Make dir
#
os.makedirs(log_path, exist_ok=True)
os.makedirs(dataset_path, exist_ok=True)
os.makedirs(result_path, exist_ok=True)


# /content/drive/MyDrive/2021/100_GoogleBrain_VentilatorPressurePrediction/datasets
#log_path = '/content/drive/MyDrive/2021/100_GoogleBrain_VentilatorPressurePrediction/log'

In [ ]:
#os.path.exists(google_mount_path)
#from google.colab import drive
#drive.mount(google_mount_path)

# Read data

In [ ]:

if(file_override):
  train = pd.read_csv(f"{path}/train.csv")
  test = pd.read_csv(f"{path}/test.csv")
  submission = pd.read_csv(f'{path}/sample_submission.csv')



# confirm linearity of time_step

In [ ]:
if visualize:
    time_step_diff_limit = 0.04
    non_liner_timestep_breath_ids = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        diff_se = grp["time_step"].diff()
        diff_chk = diff_se[diff_se > time_step_diff_limit]
        if len(diff_chk) != 0:
            non_liner_timestep_breath_ids.append(k)
#
#print(non_liner_timestep_breath_ids)
## results are following:
## [803, 2327, 3178, 4199, 5830, 10277, 11502, 13238, 15803, 16315, 16634, 18117, 18600, 24127, 25397, 28189, 28942, 30181, 32296, 36128, 36175, 37711, 38237, 38415, 39045, 39722, 42317, 42988, 43344, 44245, 45197, 46324, 49849, 53877, 54129, 55244, 55851, 61454, 64662, 67422, 67748, 72104, 74766, 76037, 78768, 79105, 80375, 87127, 87776, 89084, 91883, 93186, 98677, 102063, 104001, 106034, 107067, 109693, 111439, 112027, 115588, 119689, 120878, 121135, 125136]

# visualize non linearity time_step (直線でないtime_stepの可視化)

In [ ]:
if visualize:
    non_liner_timestep_df = train[train["breath_id"].isin(non_liner_timestep_breath_ids)]
    fig = go.Figure()
    for k,grp in non_liner_timestep_df.groupby("breath_id"):
        grp = grp.reset_index(drop=True)
        fig.add_trace(go.Scatter(x=grp.index, y=grp["time_step"], mode='lines', name=k))
    fig.show()

# visualize linearity time_step (直線なtime_stepの可視化)

おっと、傾きが色々とある！
計測機器側の計測インターバルがずれるとは思えないので、一回のサイクルを80分割したのだと思う。

In [ ]:
if visualize:
    liner_timestep_df = train[~train["breath_id"].isin(non_liner_timestep_breath_ids)]
    fig = go.Figure()
    for k,grp in tqdm(liner_timestep_df[:80*10000].groupby("breath_id")):
        grp = grp.reset_index(drop=True)
        fig.add_trace(go.Scatter(x=grp.index, y=grp["time_step"], mode='lines', name=k))
    fig.show()

# find minus pressure data (ターゲット値ですが)にマイナス値を持つデータを探す

In [ ]:
if visualize:
    minus_pressure_breath_ids = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        m = grp["pressure"].min()
        if m < 0:
            minus_pressure_breath_ids.append(k)
# print(minus_pressure_breath_ids)
# [542, 851, 3928, 7949, 11216, 13594, 16599, 19236, 20075, 22164, 23710, 27195, 27731, 30127, 36474, 40431, 40753, 43103, 43630, 44309, 45099, 45681, 45877, 46018, 46020, 46486, 47325, 49376, 49941, 50459, 52137, 53057, 54206, 56152, 56760, 57119, 58835, 59101, 60949, 65596, 67080, 67788, 70753, 71461, 72011, 74977, 77803, 83713, 85391, 86508, 90584, 91132, 91464, 92955, 94037, 97520, 98041, 98080, 101951, 106703, 108406, 109424, 109761, 110499, 111419, 112036, 113323, 113639, 118131, 119582, 120445, 121570, 124575]

# visualize minus pressure data (負のpressureを持つデータを可視化)

In [ ]:
if visualize:
    minus_pressure_df = train[train["breath_id"].isin(minus_pressure_breath_ids)]
    minus_pressure_df_plotly = pd.melt(minus_pressure_df,id_vars=["time_step","breath_id"], value_vars=["pressure"])
    fig = px.line(minus_pressure_df_plotly, x="time_step" , y="value",color = "variable",line_group ="breath_id")
    for line in fig.data:
        line['line']['color']='rgba(0, 0, 255, 0.1)'
    fig.show()

# count steps of u_out = 1   (排気段階)のstep数をtrain/testデータで確認

In [ ]:


if visualize3:
    u_out_open_step_counts = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[1]
        u_out_open_step_counts.append(count)
    
    u_out_open_step_counts_test = list()
    for k, grp in tqdm(test.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[1]
        u_out_open_step_counts_test.append(count)

# visualize by histgram counts of u_out = 1 (排気段階)のstep数の分布を可視化
48 ~ 51にまとまっている。裾のは、55まで。

In [ ]:
if visualize3: 
    fig = px.histogram(x=u_out_open_step_counts,nbins=8)
    fig.update_layout(title="u_out = 1 count histgram in train")
    fig.show()



In [ ]:
if visualize3:
    fig = px.histogram(x=u_out_open_step_counts_test,nbins=8)
    fig.update_layout(title="u_out = 1 count histgram in test")
    fig.show()

# data count of counts over 52 of u_out = 1 (排気段階)のstep数が52以上のデータ数

In [ ]:
if visualize3: 
    u_out_open_step_counts_over52 = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[1]
        if count > 51:
            u_out_open_step_counts_over52.append(count)
    len(u_out_open_step_counts_over52)

# visualize pressuer of u_out = 1

In [ ]:
if visualize3:
    train_visualize3 = train[:80*1000]
    train_visualize3 = train_visualize3[train_visualize3["u_out"] == 1]
    pressure_df_uout = pd.melt(train_visualize3,id_vars=["time_step","breath_id"], value_vars=["pressure"])
    fig = px.line(pressure_df_uout, x="time_step" , y="value",color = "variable",line_group ="breath_id")
    for line in fig.data:
        line['line']['color']='rgba(0, 0, 255, 0.05)'
    fig.show()
    

# count steps of u_out = 0 step

In [ ]:
if visualize3:
    u_out_close_step_counts = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()
        #print("count : {}".format(count))
        
        count = grp.groupby("u_out")["id"].count()[0]
        u_out_close_step_counts.append(count)
    
    u_out_close_step_counts_test = list()
    for k, grp in tqdm(test.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[0]
        u_out_close_step_counts_test.append(count)

In [ ]:
if visualize3:
    u_out_close_step_counts = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()#[0]
    print(count)
        #u_out_close_step_counts.append(count)

In [ ]:
if visualize3: 
    fig = px.histogram(x=u_out_close_step_counts,nbins=8)
    fig.update_layout(title="u_out = 0 count histgram in train")
    fig.show()

In [ ]:
if visualize3:
    fig = px.histogram(x=u_out_close_step_counts_test,nbins=8)
    fig.update_layout(title="u_out = 0 count histgram in test")
    fig.show()

# Utilitys

data_cleanの中で、直線性の無いデータ、負のpressureを持つデータ、u_out = 1のstep数52以上あるデータを削除。<br>

In [ ]:
def data_clean(df):
    ## pickup ignore breath id
    ignore_breath_ids = set()
    
    
    time_step_diff_limit = 0.04
    for k, grp in tqdm(df.groupby("breath_id")):
        
        ## ignore non liner time_step data
        diff_se = grp["time_step"].diff()
        diff_chk = diff_se[diff_se > time_step_diff_limit]
        if len(diff_chk) != 0:
            ignore_breath_ids.add(k)
            
        ## ignor negative pressure data
        m = grp["pressure"].min()
        if m < 0:
            ignore_breath_ids.add(k)
            
        ## ignore (u_out = 0 step) < 29
        count0 = grp.groupby("u_out")["id"].count()[0]
        if count0 < 29:
            ignore_breath_ids.add(k)
    
        count1 = grp.groupby("u_out")["id"].count()[1]
        if count1 > 51:
            ignore_breath_ids.add(k)
    
    df = df[~df["breath_id"].isin(np.array(list(ignore_breath_ids)))]
    return df

def change_type(df):
    #df['R'] = df['R'].astype(str)
    #df['C'] = df['C'].astype(str)
    return df

def str2dumy(df, target_col):
    #####################################
    # R or C
    #
    _df = pd.get_dummies(df[target_col])
    new_col = []
    for col in _df.columns:
        new_col.append("{}{}".format(target_col, col))
    print("new_col:{}".format(new_col))
    _df.columns = new_col
    df = pd.concat([df, _df], axis=1)
    # df = df.drop([target_col])
    df = df.drop(columns=target_col)
    return df

def add_features(df):
    print("Rolling......")
    df['u_in_roll10'] = df.groupby('breath_id')['u_in'].rolling(10, min_periods=1).mean().values
    df['u_in_roll20'] = df.groupby('breath_id')['u_in'].rolling(20, min_periods=1).mean().values
    print("...............................ok")
    print("Diff......")
    df['u_in_diff1'] = df.groupby('breath_id')['u_in'].diff(1)
    df['u_in_diff2'] = df.groupby('breath_id')['u_in'].diff(2)
    print("...............................ok")


    
    print("Cumsum, shift......")
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['u_in_cumsum'] = df.groupby('breath_id')['u_in'].cumsum()
    print("...............................ok")

    print("Shift......")
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    print("...............................ok")

    print("IN, MAX......")
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("...............................ok")

    print("IN, MAX......")
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    print("...............................ok")


    print("Cross......")
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    print("...............................ok")

    print("Category......")
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = str2dumy(df, 'R__C')
    df = str2dumy(df, 'R')
    df = str2dumy(df, 'C')
    df = str2dumy(df, 'u_out')
    print("...............................ok")

    df = df.fillna(0)
    return df


def tf_tpu_or_gpu_or_cpu():
    tpu = None
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        return "tpu"

    elif tf.test.is_gpu_available():
        strategy = tf.distribute.get_strategy()
        print('Running on GPU')
        return "gpu"

    else:
        strategy = tf.distribute.get_strategy()
        print('Running on CPU')
        return "cpu"


# 排気時のみのデータ作成

u_outのstep数最大(今回は51)でcut。短い場合は、一番最後のデータをコピーして51に合わせる。

In [ ]:
def exhaust_mode_df(df):
    grp_len = int(51)
    new_df = pd.DataFrame()
    for k, grp in df.groupby("breath_id"):
        tmp_df = grp[grp["u_out"] == 1]
        rowno = tmp_df.shape[0]
        for l in range(grp_len - rowno):
            tmp_df = tmp_df.append(grp.tail(1),ignore_index=True)
        new_df = new_df.append(tmp_df,ignore_index=True)
    return new_df

In [ ]:
def uout0_mode_df(df):
    grp_len = int(32)
    new_df = pd.DataFrame()
    for k, grp in df.groupby("breath_id"):
        tmp_df = grp[grp["u_out"] == 0]
        rowno = tmp_df.shape[0]
        if "pressure" in tmp_df.columns:
            for l in range(grp_len - rowno):
                tmp_df = tmp_df.append({""},ignore_index=True)
        else:
            for l in range(grp_len - rowno):
                tmp_df = tmp_df.append(grp.tail(1),ignore_index=True)
        new_df = new_df.append(tmp_df,ignore_index=True)
    return new_df

# データ読み込みとconfig設定

In [ ]:
#path = '/content/drive/MyDrive/2021/100_GoogleBrain_VentilatorPressurePrediction/datasets'
train = pd.read_csv(f"{path}/train.csv")
test = pd.read_csv(f"{path}/test.csv")
submission = pd.read_csv(f'{path}/sample_submission.csv')

# Debug モード時の処理
データ数を削減<br>
In use_trained_model case, use new data.

In [ ]:
if debug_mode:
    if use_trained_model:
        train = train[80*debug_data_count:80*debug_data_count*2]
        test = test[80*debug_data_count:80*debug_data_count*2]
    else:
        train = train[:80*debug_data_count]
        test = test[:80*debug_data_count]

# apply utilitys for data
データの追加と削除がメイン

In [ ]:
# exhaust_mode_df() do not work collectry
u_out1_only = False

if(file_override or (not os.path.exists('{}/train_full_feat.csv'.format(dataset_path)))):

  ## trainのみ、data_cleanを実施
  #
  train.to_csv('{}/train_origin.csv'.format(dataset_path))

  train = data_clean(train)  ## time_stepがリニア出ないものは削除
  train.to_csv('{}/train_clean.csv'.format(dataset_path))


  if u_out1_only:
      train = exhaust_mode_df(train)

  train = add_features(train)
  train = change_type(train)
  train.to_csv('{}/train_full_feat.csv'.format(dataset_path))

  print(train.head(10))

if(file_override or (not os.path.exists('{}/test_full_feat.csv'.format(dataset_path)))):
  if u_out1_only:
      test = exhaust_mode_df(test)
  test = add_features(test)
  test = change_type(test)
  test.to_csv('{}/test_full_feat.csv'.format(dataset_path))


In [ ]:
if(file_override or (not os.path.exists('{}/sample_submission.csv'.format(dataset_path)))):
    submission.to_csv('{}/sample_submission.csv'.format(dataset_path))

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
train = pd.read_csv(f"{dataset_path}/train_full_feat.csv")
test = pd.read_csv(f"{dataset_path}/test_full_feat.csv")
submission = pd.read_csv(f'{dataset_path}/sample_submission.csv')

# choose useful data for train and test and create target data
targetデータの作成と、不要なcolumnの削除。

In [ ]:
if u_out1_only:
    targets = train[['pressure']].to_numpy().reshape(-1, 51)
    train = train.drop(['pressure', 'id', 'breath_id','u_out'], axis=1)
    test = test.drop(['id', 'breath_id','u_out'], axis=1)
else:
    targets = train[['pressure']].to_numpy().reshape(-1, 80)
    train = train.drop(['pressure', 'id', 'breath_id'], axis=1)
    test = test.drop(['id', 'breath_id'], axis=1)


# 入力データのデータそのものの変形

### RobustScalerについて
入力データの分布に応じで変形。外れ値に強くしたり、正規分布に近くしたりできる。<br>
https://helve-blog.com/posts/python/scikit-learn-feature-scaling/
<br><br>
### numpy bload cast error について
原因は、trainとtestのデータの幅が違ったから。<br>
RobustScalerのfitで値を変換するその内容を決定し、transformで値の変更を実施する。<br>
そのため、fitしたときのcolumnの幅と、transformするときのcolumnの幅が揃わないとだめ。<br>
https://www.headboost.jp/numpy-array-broadcasting/
<br><br>

In [ ]:
#scaler = RobustScaler()
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# データ形状をTF向けに変更
breath_idごとのデータになるように整形。<br>

In [ ]:
if u_out1_only:
    train = train.reshape(-1, 51, train.shape[-1])
    test = test.reshape(-1, 51, test.shape[-1])
else:
    train = train.reshape(-1, 80, train.shape[-1])
    test = test.reshape(-1, 80, test.shape[-1])

# run tf function

In [ ]:
def run_tf_blstm(epoch=int(50),batch_size=int(1024),train=None,test=None,targets=None):
    
    print("train.shape[-2:]:{}".format(train.shape[-2:]))
    print("train.shape  :{}".format(train.shape))
    print("targets.shape:{}".format(targets.shape))
        
    #kf = KFold(n_splits=5, shuffle=True, random_state=2000)
    kf = KFold(n_splits=5, shuffle=True)
    test_preds = []
    test_history = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)

        # TensorBoard setting
        tensorboard_callback = tf.keras.callbacks.TensorBoard("{}-{}".format(log_path, fold+1), histogram_freq=1)

        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        model = keras.models.Sequential([
            #keras.layers.Embedding(input_dim=train.shape[-2:], output_dim=300, mask_zero=True),
            keras.layers.Input(shape=train.shape[-2:]),
            keras.layers.Bidirectional(keras.layers.LSTM(300, return_sequences=True)),          
            keras.layers.Bidirectional(keras.layers.LSTM(250, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)),
            keras.layers.Dense(50, activation='selu'),
            keras.layers.Dense(1),
        ])
        model.compile(optimizer="adam", loss="mae")    
        
        scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/batch_size), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=1)

        #es = EarlyStopping(monitor="val_loss", patience=15, verbose=1, mode="min", restore_best_weights=True)

        history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epoch, batch_size=batch_size, callbacks=[lr, tensorboard_callback])
        test_history.append(history.history)
        print("model_saveing.........")
        model.save('{}/model_save_fold{}'.format(log_path, fold+1))
        #test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

        result = model.predict(test)
        test_preds.append(result.squeeze())
    
    return test_preds, test_history

def run_tf_lstm(epoch=int(50),batch_size=int(1024),train=None,test=None,targets=None):
    #kf = KFold(n_splits=5, shuffle=True, random_state=2000)
    kf = KFold(n_splits=5, shuffle=True)
    test_preds = []
    test_history = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        model = keras.models.Sequential([
            #keras.layers.Embedding(input_dim=train.shape[-2:], output_dim=300, mask_zero=True),
            keras.layers.Input(shape=train.shape[-2:]),
            keras.layers.LSTM(300, return_sequences=True),
            keras.layers.LSTM(250, return_sequences=True),
            keras.layers.LSTM(150, return_sequences=True),
            keras.layers.LSTM(100, return_sequences=True),
            keras.layers.Dense(50, activation='selu'),
            keras.layers.Dense(1),
        ])
        model.compile(optimizer="adam", loss="mae")    
        
        scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/batch_size), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=1)

        #es = EarlyStopping(monitor="val_loss", patience=15, verbose=1, mode="min", restore_best_weights=True)

        history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epoch, batch_size=batch_size, callbacks=[lr])
        test_history.append(history.history)
        print("model_saveing.........")
        model.save(f'model_save_fold{fold+1}')
        result = model.predict(test)
        #print(result.shape) #(50300, 80, 1)
        #print(result.squeeze().shape)  #(50300, 80)
        #print(result.squeeze().reshape(-1,1).shape) #(4024000, 1)
        #test_preds.append(result.squeeze().reshape(-1, 1).squeeze())
        test_preds.append(result.squeeze())
    
    return test_preds, test_history

# Train と Test

## 戦略
* full randamでロバスト性を狙う。外れ値は外したデータなので、randamで振られても安定するはず。
* modelをsave。後で答えとどれだけずれているか見る。
* <s>use only u_out = 1 data.</s>

A.I<br>
check 

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
!nvidia-smi

In [ ]:
EPOCH = 20
#EPOCH = 200
#BATCH_SIZE = 2048
BATCH_SIZE = 1024*2
test_preds = list()

device = tf_tpu_or_gpu_or_cpu()

if use_trained_model:
    pass
else:
    if device == "cpu" :
        test_preds,history = run_tf_blstm(epoch=EPOCH,batch_size=BATCH_SIZE,train=train,test=test,targets=targets)

    elif device == "gpu":
        test_preds,history = run_tf_blstm(epoch=EPOCH,batch_size=BATCH_SIZE,train=train,test=test,targets=targets)
    elif device == "tpu":
        try:
            with tpu_strategy.scope():
                test_preds,history = run_tf_blstm(epoch=EPOCH,batch_size=BATCH_SIZE,train=train,test=test,targets=targets)
        except :
            print('Error')

# re shape prediction results

In [ ]:
if use_trained_model:
    pass
else:
    print("make pread......")
    preds = np.mean(test_preds, axis=0).tolist()

In [ ]:
print("test_preds:{}".format(len(test_preds)))
print("test_preds[0]:{}".format(test_preds[0].shape))

print("preds:{}".format(len(preds)))
#print("test_preds:{}".format(test_preds.shape))
print("preds:{}".format(len(preds[0])))

preds_array = np.array(preds).reshape(-1)
print("preds_array:{}".format(preds_array.shape))

# re read test.csv and add pred results to "pressure"

In "u_out1_only = False" case, do not need to run following function.<br>
Just only for "u_out1_only = True" case.

In [ ]:
def add_pressure_data(df,preds):
    index = 0
    new_df = pd.DataFrame()
    for k,grp in df.groupby("breath_id"):
        u_out_len = len(grp[grp["u_out"] == 1])
        if u_out_len < 52:
            grp.loc[grp["u_out"] == 1,"pressure"] = preds[index][:u_out_len]
        else:
            out_preds = preds[index]
            for l in range(u_out_len - 51):
                out_preds = out_preds.append(out_preds[-1])
            grp.loc[grp["u_out"] == 1,"pressure"] = out_preds
        new_df = new_df.append(grp,ignore_index=True)
        index += 1
    return new_df

# In non debug mode, create submission data

In [ ]:
if debug_mode:
    pass
else:
    submission["pressure"] = preds_array
    submission.to_csv('submission_lstm.csv', index=False)
    submission.to_csv('submission.csv', index=False)
    print("make submission.csv........ ok")

In [ ]:
 !zip -r file.zip 'submission_lstm.csv'

In [ ]:
from IPython.display import FileLink
print(FileLink(r'submission_lstm.csv'))
FileLink(r'submission_lstm.csv')

# Load Pre Train model

In [ ]:
#if use_trained_model:
if(0):
    model = tf.keras.models.load_model('/content/drive/MyDrive/2021/100_GoogleBrain_VentilatorPressurePrediction/log/mylstm_v3.2_lite/model_save_fold2')
    result = model.predict(train)
    result = result.squeeze()

# Compare model result and train data, And visualize

In [ ]:
## res and tgt is numpy array
def result_visual(result, targets, view_txt):
  count_l = 0
  count_s = 0
  for res, tgt in zip(result,targets):
      abs_diff =  np.sum(np.abs(res - tgt))
      
      if abs_diff > 500:
          count_l += 1
          if(count_l > 10):
              continue

          df = pd.DataFrame(res)
          df = pd.concat([df, pd.DataFrame(tgt)], axis=1)
          df.columns = ["res","tgt"]
          df["id"] = df.index
          df = pd.melt(df,id_vars=["id"])
          plt.figure()
          fig = sns.lineplot(data=df, x='id', y="value", hue="variable")
          fig.set_title(f"{view_txt} >> ** abs_diff : {abs_diff}")
      elif abs_diff < 30:
          count_s += 1
          if(count_s > 10):
              continue
          df = pd.DataFrame(res)
          df = pd.concat([df, pd.DataFrame(tgt)], axis=1)
          df.columns = ["res","tgt"]
          df["id"] = df.index
          df = pd.melt(df,id_vars=["id"])
          plt.figure()
          fig = sns.lineplot(data=df, x='id', y="value", hue="variable")
          fig.set_title(f"{view_txt} >> oo abs_diff : {abs_diff}")

def result_dict_visual(result_dict, targets, view_txt):
  count_l = 0
  count_s = 0
  model_name_list = list(result_dict.keys())
  #print(model_name_list)
  #print(result_dict[model_name_list[0]].shape)
  
  for idx in tqdm(range(result_dict[model_name_list[0]].shape[0])):
    
    gt  = pd.DataFrame(targets[idx, :])
    gt.columns = ["gt"]
    gt["id"] = gt.index

    # OK
    # メモリが解放される
    plt.clf()
    plt.close()

    fig = plt.figure(figsize=(10.0, 10.0))
    ax = fig.add_subplot(111)

    ax = sns.lineplot(data=gt, x='id', y='gt', ax=ax, label='gt')

    abs_diff_list = []
    for model_name in model_name_list:
      pred = result_dict[model_name]
      pred = pd.DataFrame(pred[idx, :])
      pred.columns = ["pred"]
      pred["id"] = pred.index

      _abs_diff =  np.sum(np.abs(gt['gt'].values - pred['pred'].values))
      abs_diff_list.append(_abs_diff)
      
      abs_diff = np.mean(abs_diff_list)
      # print(abs_diff)
      
      ax = sns.lineplot(data=pred, x='id', y='pred', ax=ax, label=model_name)

    if(abs_diff < 30 or 500 < abs_diff):
      ax.set_title(">> ** abs_diff : {: 8.4f}".format(abs_diff))
      # print("fig saveing .............")
      plt.savefig('{}/pred_gt_diff{:08.4f}_idx{:06d}.png'.format(result_path, abs_diff, idx))

    plt.close(fig)

In [ ]:
#################################################
# find model
#
save_model_list = glob.glob('{}/*model*'.format(log_path))
pprint.pprint(save_model_list)

In [ ]:
#################################################
# laod save model
#
result_dict = {}
for model_path in save_model_list:
  model_name = model_path.split('/')[-1]
  print(">>>>>>>> {} >>>>>>>>>>>".format(model_name))
  model = tf.keras.models.load_model(model_path)
  _result = model.predict(train)
  _result = _result.squeeze()
  result_dict[model_name] = _result

In [ ]:
#################################################
# visual model result
#
#result_dict_visual(result_dict, targets, model_name)
result_dict_visual(result_dict, targets, model_name)


#for model_name, result in result_dict.items():
#  print(">>>>>>>> {} >>>>>>>>>>>".format(model_name))
#  #result_visual(result, targets, model_name)


# 